In [2]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np
from ensemble_boxes import *

# from ensemble_boxes import *

In [2]:
import wandb
print(wandb.__version__)

0.10.30


In [3]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/swin/cascade_mask_rcnn_swin_base_patch4_window7_mstrain_480-800_giou_4conv1f_adamw_3x_coco.py')
PREFIX = '../../input/data/'

epoch = 15

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 4

cfg.seed=2020
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/swin'

# cfg.model.roi_head.bbox_head.num_classes = 11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'epoch_{epoch}.pth')

In [4]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [5]:
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

2021-05-17 04:19:51,355 - mmdet - INFO - load model from: /opt/ml/code/mmdetection_trash/configs/_base_/swin_weights/swin_base_patch4_window7_224.pth
2021-05-17 04:19:52,367 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: norm.weight, norm.bias, head.weight, head.bias, layers.0.blocks.1.attn_mask, layers.1.blocks.1.attn_mask, layers.2.blocks.1.attn_mask, layers.2.blocks.3.attn_mask, layers.2.blocks.5.attn_mask, layers.2.blocks.7.attn_mask, layers.2.blocks.9.attn_mask, layers.2.blocks.11.attn_mask, layers.2.blocks.13.attn_mask, layers.2.blocks.15.attn_mask, layers.2.blocks.17.attn_mask

missing keys in source state_dict: norm0.weight, norm0.bias, norm1.weight, norm1.bias, norm2.weight, norm2.bias, norm3.weight, norm3.bias

Use load_from_local loader


In [6]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05)

[                                 ] 1/837, 1.5 task/s, elapsed: 1s, ETA:   561s/opt/conda/lib/python3.7/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 6.1 task/s, elapsed: 138s, ETA:     0s

In [7]:
output[0][0]

[array([[1.6531673e+02, 7.6644920e+01, 2.5394872e+02, 1.6636185e+02,
         8.0189362e-02]], dtype=float32),
 array([[1.6488878e+02, 7.7089928e+01, 2.5461528e+02, 1.6618219e+02,
         1.4444630e-01],
        [1.8398605e+02, 3.5835867e+02, 2.0480710e+02, 3.9610355e+02,
         6.4115703e-02],
        [1.8455896e+02, 3.5895508e+02, 2.0547046e+02, 3.9571844e+02,
         5.0734128e-03],
        [1.6479523e+02, 7.7343689e+01, 2.5572623e+02, 1.6613284e+02,
         1.2491539e-03]], dtype=float32),
 array([], shape=(0, 5), dtype=float32),
 array([], shape=(0, 5), dtype=float32),
 array([], shape=(0, 5), dtype=float32),
 array([], shape=(0, 5), dtype=float32),
 array([], shape=(0, 5), dtype=float32),
 array([], shape=(0, 5), dtype=float32),
 array([[1.7342540e+02, 2.3564621e+02, 4.6179681e+02, 4.7163132e+02,
         9.9930787e-01],
        [1.7417262e+02, 2.3736305e+02, 4.6392999e+02, 4.7144141e+02,
         1.3242973e-02]], dtype=float32),
 array([], shape=(0, 5), dtype=float32),
 arr

In [8]:
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 11
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    out = out[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'swin_cascade_submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,0 0.08018936 165.31673 76.64492 253.94872 166....,batch_01_vt/0021.jpg
1,1 0.816722 0.028137207 248.58307 301.959 458.7...,batch_01_vt/0028.jpg
2,1 0.9992367 67.25845 168.12982 374.12405 378.7...,batch_01_vt/0031.jpg
3,2 0.9472642 297.08646 157.00125 364.1107 228.4...,batch_01_vt/0032.jpg
4,0 0.06883724 21.38771 133.05128 388.1521 278.8...,batch_01_vt/0070.jpg


In [7]:
####################### 여기부터 wbf #########################

submission_files = ['/opt/ml/code/mmdetection_trash/work_dirs/swin/swin_cascade_submission_15.csv', '/opt/ml/code/mmdetection_trash/work_dirs/swin/swin_cascade_submission_11.csv', '/opt/ml/code/mmdetection_trash/work_dirs/swin/output_5090.csv', '/opt/ml/code/mmdetection_trash/work_dirs/swin/output_5110.csv', '/opt/ml/code/mmdetection_trash/work_dirs/swin/output.csv']
submission_df = [pd.read_csv(file) for file in submission_files]
image_ids = submission_df[0]['image_id'].tolist()
annotation = '/opt/ml/input/data/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [13]:
prediction_strings = []
file_names = []
iou_thr = 0.5

for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        if len(predict_list)==0 or len(predict_list)==1:
            continue
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    # print(labels_list[:10])   
    if len(boxes_list):
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, iou_thr=iou_thr)
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

[ 8.  2.  1. 10.  1.  8.  0.  5.  1.  6.]
[8. 8. 8. 1. 8. 8. 8. 8. 6. 8.]
[1. 1. 3. 8. 6. 2.]
[8. 2. 8. 8. 2. 6. 2. 2. 8. 6.]
[8. 8. 8. 8. 7. 8. 2. 8. 1. 8.]
[8. 8. 8. 8. 8. 8. 1. 1. 8. 8.]
[8. 8. 8. 8. 8. 8. 8. 8. 8. 8.]
[6. 1. 1. 1. 6.]
[8. 8. 8. 8. 8. 8. 8. 1. 2. 6.]
[8. 1. 8. 2. 2. 8. 1. 8. 2. 1.]
[2. 1. 3.]
[8. 8.]
[5. 1. 1. 1. 6. 1. 1. 1. 1. 1.]
[6. 4. 5.]
[10.  1.  1.  0.]
[10. 10. 10.  5.  6. 10.]
[2. 6. 1. 1.]
[8. 2. 8. 3. 1. 1. 6. 6. 8. 2.]
[6. 1. 1. 1. 6. 1. 1. 3. 2. 6.]
[8. 8. 8. 2. 2. 2. 2. 2. 8. 2.]
[6. 8. 6. 8. 6. 8. 2. 2. 2. 2.]
[2. 8. 1. 1. 1. 1. 1. 6. 1. 3.]
[3. 3. 6. 2. 6. 3. 3. 3. 3. 3.]
[8. 8. 8. 2. 6. 8. 2. 8. 8. 0.]
[8. 6. 1.]
[8. 6. 1. 4. 6. 1.]
[7. 1. 1. 6. 6. 1. 1. 3. 1. 1.]
[3. 3. 6. 6. 1. 6. 1. 0. 7. 3.]
[8. 8. 8. 8. 8. 2. 2. 6. 8. 8.]
[8. 2. 2. 8. 2. 8. 2. 1. 2. 2.]
[3. 4. 1. 3. 2. 1. 8. 5. 6. 2.]
[1. 8. 2. 7. 3.]
[6.]
[6. 8.]
[2. 2. 1. 1. 6. 1.]
[3. 6. 1. 8. 1. 1. 2. 1. 1. 1.]
[8. 1. 2. 1. 1. 2. 6. 2. 6. 1.]
[6. 1. 5. 6. 1. 6.]
[1. 6.]
[3. 2. 6. 1. 2. 8. 1

In [14]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('submission_jkw_ensemble_wbf_2.csv')

submission.head()

,PredictionString,image_id
0,8 0.8348873853683472 173.9508819580078 233.958...,batch_01_vt/0021.jpg
1,8 0.833134651184082 157.7088623046875 58.56629...,batch_01_vt/0028.jpg
2,1 0.8181977272033691 70.27095031738281 168.971...,batch_01_vt/0031.jpg
3,8 0.7131975293159485 32.29524230957031 123.718...,batch_01_vt/0032.jpg
4,8 0.8382120728492737 158.3148193359375 350.059...,batch_01_vt/0070.jpg


In [10]:
submission.head(20)

,PredictionString,image_id
0,8.0 0.8348873853683472 173.9508819580078 233.9...,batch_01_vt/0021.jpg
1,8.0 0.8083720803260803 457.8204345703125 348.6...,batch_01_vt/0028.jpg
2,1.0 0.8181977272033691 70.27095031738281 168.9...,batch_01_vt/0031.jpg
3,8.0 0.6124548316001892 32.068111419677734 143....,batch_01_vt/0032.jpg
4,8.0 0.8346942663192749 43.47178268432617 59.41...,batch_01_vt/0070.jpg
5,8.0 0.8353999257087708 122.03097534179688 246....,batch_01_vt/0076.jpg
6,8.0 0.8314574360847473 192.34588623046875 87.6...,batch_01_vt/0078.jpg
7,1.0 0.6283218264579773 212.51910400390625 196....,batch_01_vt/0096.jpg
8,8.0 0.8354833126068115 136.8455352783203 274.6...,batch_01_vt/0101.jpg
9,8.0 0.6226121783256531 0.06710121780633926 118...,batch_01_vt/0122.jpg


In [2]:
import pandas as pd
import numpy as np
PredictionString = pd.read_csv('/opt/ml/code/mmdetection_trash/work_dirs/swin/swin_cascade_submission_11.csv')['PredictionString']
annotation = []
ids = 0
for image_id,bboxs in enumerate(PredictionString):
    bboxs = bboxs.strip().split(' ')
    bboxs = [float(i) for i in bboxs]
    bboxs = [bboxs[i:i + 6] for i in range(0, len(bboxs), 6)]
    for bbox in bboxs:
        temp = dict()
        temp['id'] = str(ids)
        temp['image_id'] = str(image_id)
        temp['category_id'] = str(int(bbox[0]))
        temp['segmentation'] = [[str(1)]]
        temp['area'] = 1
        temp['bbox'] = [str(i) for i in list(np.round(bbox[2:], 1))]
        if len(bbox)!=6:
            print(bbox)
        temp['iscrowd'] = str(0)
        annotation.append(temp)
        ids+=1

In [5]:
annotation[:10]

[{'id': '0',
  'image_id': '0',
  'category_id': '1',
  'segmentation': [['1']],
  'area': 1,
  'bbox': ['166.1', '76.2', '253.0', '166.0'],
  'iscrowd': '0'},
 {'id': '1',
  'image_id': '0',
  'category_id': '2',
  'segmentation': [['1']],
  'area': 1,
  'bbox': ['165.7', '76.0', '252.8', '166.4'],
  'iscrowd': '0'},
 {'id': '2',
  'image_id': '0',
  'category_id': '5',
  'segmentation': [['1']],
  'area': 1,
  'bbox': ['0.0', '335.1', '78.4', '449.2'],
  'iscrowd': '0'},
 {'id': '3',
  'image_id': '0',
  'category_id': '6',
  'segmentation': [['1']],
  'area': 1,
  'bbox': ['0.0', '336.9', '78.4', '447.8'],
  'iscrowd': '0'},
 {'id': '4',
  'image_id': '0',
  'category_id': '6',
  'segmentation': [['1']],
  'area': 1,
  'bbox': ['166.2', '75.8', '253.1', '166.2'],
  'iscrowd': '0'},
 {'id': '5',
  'image_id': '0',
  'category_id': '8',
  'segmentation': [['1']],
  'area': 1,
  'bbox': ['174.8', '233.2', '461.7', '470.2'],
  'iscrowd': '0'},
 {'id': '6',
  'image_id': '0',
  'category

In [10]:
import json
with open('/opt/ml/input/data/train.json', 'r') as f:
    json_data = json.load(f)